<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt

fatal: destination path 'StockSellRulesChecker' already exists and is not an empty directory.


In [62]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

import time

In [71]:
eps_5year_over5pct = False
eps_qoq_over5pct = False
eps_yoy_over5pct = False
sales_5years_over5pct = False
sales_qoq_over5pct = False

filters = []
if eps_5year_over5pct:
  filters.append('fa_eps5years_o5')
if eps_qoq_over5pct:
  filters.append('fa_epsqoq_o5')
if eps_yoy_over5pct:
  filters.append('fa_epsyoy_o5')
if sales_5years_over5pct:
  filters.append('fa_sales5years_o5')
if sales_qoq_over5pct:
  filters.append('fa_salesqoq_o5')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [82]:
print(stock_list)

No.  | Ticker | Perf Week | Perf Month | Perf Quart | Perf Half | Perf Year | Perf YTD | Volatility W | Volatility M | Recom | Avg Volume | Rel Volume | Price | Change  | Volume    
---- | ------ | --------- | ---------- | ---------- | --------- | --------- | -------- | ------------ | ------------ | ----- | ---------- | ---------- | ----- | ------- | ----------
1    | GEN    | -2.46%    | -46.02%    | -1.84%     | -14.46%   | -47.05%   | 14.71%   | 8.33%        | 16.74%       | 3.50  | 6.80M      | 0.64       | 0.54  | -3.76%  | 3,889,704 
2    | AAU    | -4.78%    | -33.67%    | -5.82%     | -29.87%   | 126.97%   | 32.91%   | 7.83%        | 10.74%       | -     | 5.93M      | 0.62       | 0.61  | -1.99%  | 3,279,850 
3    | GSV    | 9.05%     | -24.80%    | -11.57%    | -21.22%   | 51.50%    | -7.90%   | 6.31%        | 8.37%        | 2.00  | 7.81M      | 0.33       | 0.65  | -2.34%  | 2,300,378 
4    | TRX    | 3.52%     | -21.19%    | 5.49%      | -6.43%    | 18.48%    | 5.14%    | 5

In [83]:
def moving_average(yahoo_df, days):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=days)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())


screened_stocks = {}
for stock in tqdm.tqdm(stock_list, total=len(stock_list.data)):
  print(stock['Ticker'])
  if stock["Ticker"] == "":
    continue
  screened_stocks[stock['Ticker']] = {}
  finviz_stats = finviz.get_stock(stock['Ticker'])
  try:
      yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
  except:
    continue

  SMA200_value = moving_average(yahoo_df, days=200)
  SMA150_value = moving_average(yahoo_df, days=150)
  SMA50_value = moving_average(yahoo_df, days=50)
  SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
  SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))

  try:
    EPS_ttm = float(finviz_stats['EPS (ttm)'])
  except:
    EPS_ttm = 0

  try:
    EPS_QoQ_percent = float(finviz_stats['EPS Q/Q'].replace("%",""))
  except:
    EPS_QoQ_percent = 0

  try:
    Sales_QoQ_percent = float(finviz_stats['Sales Q/Q'].replace("%",""))
  except:
    Sales_QoQ_percent = 0

  try:
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))/100
  except:
    inst_own = 0
  
  shares_outstanding = finviz_stats['Shs Outstand']
  if 'M' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
  elif 'B' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
  elif 'T' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12
  else:
    shares_outstanding = 0

  prev_close = float(finviz_stats['Prev Close'].replace("$",""))
  week52_high, week52_low = week52_low_high(yahoo_df)
  
  screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
  screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
  screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
  screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
  screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
  screened_stocks[stock['Ticker']]['EPS_QoQ_percent'] = EPS_QoQ_percent
  screened_stocks[stock['Ticker']]['EPS_ttm'] = EPS_ttm
  screened_stocks[stock['Ticker']]['Sales_QoQ_percent'] = Sales_QoQ_percent
  screened_stocks[stock['Ticker']]['prev_close'] = prev_close
  screened_stocks[stock['Ticker']]['week52_high'] = week52_high
  screened_stocks[stock['Ticker']]['week52_low'] = week52_low
  screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
  screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
  
  # EPS TTM > $1 Rule
  if EPS_ttm >= 1.0:
    eps_ttm_rule = True
  else:
    eps_ttm_rule = False

  # EPS QoQ Yearly > 18% rule
  if EPS_QoQ_percent >= 0.18:
    eps_QoQ_yearly_rule = True
  else:
    eps_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['eps_QoQ_yearly_rule'] = eps_QoQ_yearly_rule

  # Sales QoQ Yearly > 25% rule
  if Sales_QoQ_percent >= 0.25:
    sales_QoQ_yearly_rule = True
  else:
    sales_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['sales_QoQ_yearly_rule'] = sales_QoQ_yearly_rule

  # Shares Outstanding <= 25 mil
  if shares_outstanding <= 25e6:
    shares_outstanding_rule = True
  else:
    shares_outstanding_rule = False
  screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

  # Institutional Ownership < 35%
  if 0.05 <= inst_own <= 0.35:
    inst_ownership_rule = True
  else:
    inst_ownership_rule = False
  screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

  # Positive 200d MA positive
  if SMA200_percent > 0:
      SMA200_slope_rule = True
  else:
      SMA200_slope_rule = False
  screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
      
  # 150d MA greater than 200d MA
  if SMA150_value > SMA200_value:
      SMA150_greater_SMA200_rule = True
  else:
      SMA150_greater_SMA200_rule = False
  screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
      
  # 50d MA greater than 150d MA
  if SMA50_value > SMA150_value:
      SMA50_greater_SMA150_rule = True
  else:
      SMA50_greater_SMA150_rule = False
  screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
      
  # Close above 50d MA
  if prev_close > SMA50_value:
      close_greater_SMA50_rule = True
  else:
      close_greater_SMA50_rule = False
  screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
      
  # 52 week high low span rule
  if 0.75*week52_high > 1.25*week52_low:
      week52_span_rule = True
  else:
      week52_span_rule = False
  screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
  
  # Close above 52 week high - 25%
  if prev_close > 0.75*week52_high:
      close_above_52weekhigh_rule = True
  else:
      close_above_52weekhigh_rule = False
  screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    




  0%|          | 0/220 [00:00<?, ?it/s]

GEN




  0%|          | 1/220 [00:00<02:34,  1.42it/s]

AAU




  1%|          | 2/220 [00:01<02:30,  1.45it/s]

...



 99%|█████████▉| 218/220 [03:29<00:05,  2.65s/it]

PEBO




100%|█████████▉| 219/220 [03:32<00:02,  2.57s/it]

GRC




100%|██████████| 220/220 [03:34<00:00,  1.03it/s]


In [84]:
rules = [col for col in df_out.columns if "rule" in col]
df_out['Passed Tests'] = (df_out[list(rules)]).sum(1)

In [85]:
df_out.sort_values('Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,EPS_ttm,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,eps_QoQ_yearly_rule,sales_QoQ_yearly_rule,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
188,UAN,14.147681,16.050769,24.351389,175.69,59.01,30.2,-8.74,4.9,35.76,35.930000,6.0000,0.2460,11010000.0,True,True,True,True,True,True,True,True,True,True,10
112,YVR,1.744783,1.842597,2.121669,75.52,48.25,76.7,-0.51,1220.4,3.95,3.950000,1.3700,0.0560,10010000.0,True,True,True,True,True,True,True,True,True,True,10
87,SGRP,1.103986,1.211346,1.651111,70.96,14.60,25.6,0.03,-11.3,1.79,1.930000,0.6000,0.0620,21110000.0,True,False,True,True,True,True,True,True,True,True,9
121,FOR,20.091812,20.683365,22.520278,28.85,9.55,29.8,1.37,24.2,24.18,24.180000,9.6300,0.3220,48100000.0,True,True,False,True,True,True,True,True,True,True,9
146,SQBG,12.313370,14.647260,19.589028,225.47,94.31,123.9,-56.08,-5.5,30.00,35.605000,4.4500,0.2440,1650000.0,True,False,True,True,True,True,True,True,True,True,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,XM,42.383857,42.383857,42.383857,-18.34,-18.34,73.0,-0.59,23.6,35.41,55.240002,33.5700,0.0000,485830000.0,True,True,False,False,False,False,False,False,False,False,2
80,BNKD,6.396830,4.706813,2.528903,-79.44,-36.61,0.0,0.00,0.0,1.84,76.714996,1.7555,0.0000,0.0,False,False,True,False,False,False,False,False,True,False,2
83,GLDG,2.155725,2.034712,1.702500,-13.84,-3.71,0.0,0.00,0.0,1.79,2.900000,0.7400,0.0456,148730000.0,False,False,False,False,False,False,False,True,True,False,2
94,NEXT,2.443949,2.344663,2.135972,-20.12,-19.65,-54.8,-0.34,0.0,1.69,4.900000,1.1700,0.8500,117560000.0,False,False,False,False,False,False,False,False,True,False,1
